In [58]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from re import sub

In [3]:
url = "https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating"
response = requests.get(url)
soup = BeautifulSoup(response.content,"html.parser")
movie_data = soup.find_all("div",attrs={'class' : 'lister-item mode-advanced'})

# Title

In [5]:
title = movie_data[0].h3.a.text
title

'The Shawshank Redemption'

# Release Year

In [6]:
release_year = movie_data[0].h3.find('span',class_ = 'lister-item-year text-muted unbold').text
release_year = sub("[^0-9]","",release_year)
release_year

'1994'

# Runtime

In [7]:
runtime = movie_data[10].p.find('span',class_ = 'runtime').text
runtime = sub("[^0-9]","",runtime)
runtime

'178'

# Genre

In [8]:
genre = movie_data[2].p.find('span',class_ = 'genre').text
genre = sub('[^A-Za-z0-9 ]',"",genre).strip()
genre

'Action Crime Drama'

# Ratings

In [9]:
rating = movie_data[0].find('span',class_ = 'rating-rating').find('span',class_ = 'value').text
rating

'9.3'

# Meta Score

In [10]:
meta_score = movie_data[0].find('span',class_ = 'metascore favorable').text
meta_score = sub("[^0-9]","",meta_score)
meta_score

'82'

# Overview

In [11]:
overview = movie_data[0].find_all('p',class_ = 'text-muted')[1].text
overview = sub("[^a-zA-Z0-9 ]","",overview)
overview

'Over the course of several years two convicts form a friendship seeking consolation and eventually redemption through basic compassion'

# Director

In [12]:
director = movie_data[0].find('p',class_ = '').a.text
director

'Frank Darabont'

# Cast

In [13]:
entity = movie_data[0].find_all('p',class_ = '')[0].find_all('a')
cast = []
for i,j in enumerate(entity):
    cast.append(entity[i].text)
cast.pop(0)
cast

['Tim Robbins', 'Morgan Freeman', 'Bob Gunton', 'William Sadler']

# Votes

In [14]:
votes = movie_data[1].find('p',class_ = 'sort-num_votes-visible').find_all('span')[1].text
votes

'1,931,944'

# Gross

In [15]:
Gross = movie_data[8].find('p',class_ = 'sort-num_votes-visible').find_all('span',attrs={'data-value': True})[1]['data-value']
Gross

'107,928,762'

In [40]:
def fetch_cast(i):
    entity = movie_data[i].find_all('p',class_ = '')[0].find_all('a')
    cast = []
    for i,j in enumerate(entity):
        cast.append(entity[i].text)
    cast.pop(0)
    return cast

In [67]:
movie_data = soup.find_all("div",attrs={'class' : 'lister-item mode-advanced'})
movie_title = []
release_year = []
runtime = []
genre = []
imdb_rating = []
overview = []
director = []
cast = []
votes = []
Gross = []
meta_score = []
for i in range(100):
    movie_title.append(movie_data[i].h3.a.text)
    year = movie_data[i].h3.find('span',class_ = 'lister-item-year text-muted unbold').text
    release_year.append(sub("[^0-9]","",year))
    time = movie_data[i].p.find('span',class_ = 'runtime').text
    runtime.append(sub("[^0-9]","",time))
    gen = movie_data[i].p.find('span',class_ = 'genre').text
    genre.append(sub('[^A-Za-z0-9 ]',"",gen).strip())
    imdb_rating.append(movie_data[i].find('span',class_ = 'rating-rating').find('span',class_ = 'value').text)
    info = movie_data[i].find_all('p',class_ = 'text-muted')[1].text
    overview.append(sub("[^a-zA-Z0-9 ]","",info))
    director.append(movie_data[i].find('p',class_ = '').a.text)
    cast.append(fetch_cast(i))
    votes.append(movie_data[i].find('p',class_ = 'sort-num_votes-visible').find_all('span')[1].text)
    Gross.append(movie_data[8].find('p',class_ = 'sort-num_votes-visible').find_all('span',attrs={'data-value': True})[1]['data-value'])
    try:
        m_score = movie_data[i].find('span',class_ = 'metascore favorable').text
        meta_score.append(sub("[^0-9]","",m_score))
    except:
        meta_score.append(np.nan)

In [74]:
cols = ['movie_title','genre','overview','director','cast','runtime','imdb_rating','meta_score','votes','Gross','release_year']

In [75]:
df = pd.DataFrame(list(zip(movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year)),columns=cols)

In [76]:
df.head()

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
0,The Shawshank Redemption,Drama,Over the course of several years two convicts ...,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",142,9.3,82,"2,775,236","107,928,762",1994
1,The Godfather,Crime Drama,Don Vito Corleone head of a mafia family decid...,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K...",175,9.2,100,"1,931,944","107,928,762",1972
2,The Dark Knight,Action Crime Drama,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",152,9,84,"2,751,318","107,928,762",2008
3,Schindler's List,Biography Drama History,In Germanoccupied Poland during World War II i...,Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car...",195,9,95,"1,396,995","107,928,762",1993
4,12 Angry Men,Crime Drama,The jury in a New York City murder trial is fr...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John...",96,9,97,"823,379","107,928,762",1957


In [77]:
df.shape

(100, 11)

In [78]:
df.isnull().sum()

movie_title      0
genre            0
overview         0
director         0
cast             0
runtime          0
imdb_rating      0
meta_score      15
votes            0
Gross            0
release_year     0
dtype: int64

In [ ]:
movie_data = soup.find_all("div",attrs={'class' : 'lister-item mode-advanced'})
movie_title = []
release_year = []
runtime = []
genre = []
imdb_rating = []
overview = []
director = []
cast = []
votes = []
Gross = []
meta_score = []
for i in range(100):
    movie_title.append(movie_data[i].h3.a.text)
    year = movie_data[i].h3.find('span',class_ = 'lister-item-year text-muted unbold').text
    release_year.append(sub("[^0-9]","",year))
    time = movie_data[i].p.find('span',class_ = 'runtime').text
    runtime.append(sub("[^0-9]","",time))
    gen = movie_data[i].p.find('span',class_ = 'genre').text
    genre.append(sub('[^A-Za-z0-9 ]',"",gen).strip())
    imdb_rating.append(movie_data[i].find('span',class_ = 'rating-rating').find('span',class_ = 'value').text)
    info = movie_data[i].find_all('p',class_ = 'text-muted')[1].text
    overview.append(sub("[^a-zA-Z0-9 ]","",info))
    director.append(movie_data[i].find('p',class_ = '').a.text)
    cast.append(fetch_cast(i))
    votes.append(movie_data[i].find('p',class_ = 'sort-num_votes-visible').find_all('span')[1].text)
    Gross.append(movie_data[8].find('p',class_ = 'sort-num_votes-visible').find_all('span',attrs={'data-value': True})[1]['data-value'])
    try:
        m_score = movie_data[i].find('span',class_ = 'metascore favorable').text
        meta_score.append(sub("[^0-9]","",m_score))
    except:
        meta_score.append(np.nan)

In [107]:
movie_title = []
release_year = []
runtime = []
genre = []
imdb_rating = []
overview = []
director = []
cast = []
votes = []
Gross = []
meta_score = []
for i in range(1,902,100):
    url = "https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start={}&ref_=adv_nxt".format(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"html.parser")
    movie_data = soup.find_all("div",attrs={'class' : 'lister-item mode-advanced'})
    for i in range(100):
        movie_title.append(movie_data[i].h3.a.text)
        year = movie_data[i].h3.find('span',class_ = 'lister-item-year text-muted unbold').text
        release_year.append(sub("[^0-9]","",year))
        time = movie_data[i].p.find('span',class_ = 'runtime').text
        runtime.append(sub("[^0-9]","",time))
        gen = movie_data[i].p.find('span',class_ = 'genre').text
        genre.append(sub('[^A-Za-z0-9 ]',"",gen).strip())
        imdb_rating.append(movie_data[i].find('span',class_ = 'rating-rating').find('span',class_ = 'value').text)
        info = movie_data[i].find_all('p',class_ = 'text-muted')[1].text
        overview.append(sub("[^a-zA-Z0-9 ]","",info))
        director.append(movie_data[i].find('p',class_ = '').a.text)
        cast.append(fetch_cast(i))
        votes.append(movie_data[i].find('p',class_ = 'sort-num_votes-visible').find_all('span')[1].text)
        try:
            Gross.append(movie_data[i].find('p',class_ = 'sort-num_votes-visible').find_all('span',attrs={'data-value': True})[1]['data-value'])
        except:
            Gross.append(np.nan)
        try:
            m_score = movie_data[i].find('span',class_ = 'metascore favorable').text
            meta_score.append(sub("[^0-9]","",m_score))
        except:
            meta_score.append(np.nan)

In [108]:
cols = ['movie_title','genre','overview','director','cast','runtime','imdb_rating','meta_score','votes','Gross','release_year']

In [109]:
final_df = pd.DataFrame(list(zip(movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year)),columns=cols)

In [110]:
final_df.head()

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
0,The Shawshank Redemption,Drama,Over the course of several years two convicts ...,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",142,9.3,82,"2,775,236","28,341,469",1994
1,The Godfather,Crime Drama,Don Vito Corleone head of a mafia family decid...,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K...",175,9.2,100,"1,931,944","134,966,411",1972
2,The Dark Knight,Action Crime Drama,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",152,9,84,"2,751,318","534,858,444",2008
3,Schindler's List,Biography Drama History,In Germanoccupied Poland during World War II i...,Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car...",195,9,95,"1,396,995","96,898,818",1993
4,12 Angry Men,Crime Drama,The jury in a New York City murder trial is fr...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John...",96,9,97,"823,379","4,360,000",1957


In [112]:
final_df.to_csv("F:/ETL/Dataset/IMDB_1000.csv")

In [113]:
len(movie_title)

1000